test 1 based on feedback from: https://github.com/pyvista/pyvista/discussions/4851#discussioncomment-6941244

In [2]:
!pip install geovista

  Obtaining dependency information for geovista from https://files.pythonhosted.org/packages/11/2e/f172e82a856358bb7da0b102298956db85fb61cf55e9183871249c469f6e/geovista-0.4.0-py3-none-any.whl.metadata
  Obtaining dependency information for cartopy>=0.20 from https://files.pythonhosted.org/packages/41/f4/edfba7b0768930486d4f434183a500de75826aba82e6ffd4884ae2b410b9/Cartopy-0.22.0-cp310-cp310-win_amd64.whl.metadata
  Obtaining dependency information for click from https://files.pythonhosted.org/packages/00/2e/d53fa4befbf2cfa713304affc7ca780ce4fc1fd8710527771b58311a3229/click-8.1.7-py3-none-any.whl.metadata
  Obtaining dependency information for click-default-group from https://files.pythonhosted.org/packages/2c/1a/aff8bb287a4b1400f69e09a53bd65de96aa5cee5691925b38731c67fc695/click_default_group-1.2.4-py2.py3-none-any.whl.metadata
     ---------------------------------------- 0.0/222.1 kB ? eta -:--:--
     ------ ------------------------------ 41.0/222.1 kB 991.0 kB/s eta 0:00:01
     ----


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
# imports
import geovista as gv
import pyvista as pv
from geovista.common import to_cartesian
import pandas as pd
import datetime
import numpy as np

In [6]:
# functions
def points_to_mesh(points, df: pd.DataFrame):
    m = pv.PolyData(points)
    m.point_data["mass"] = df["mass (g)"].fillna(1.0).astype(np.float32)
    return m.glyph(
        geom   = pv.Sphere(0.01, theta_resolution = 32, phi_resolution = 32),
        scale  = "mass",
        factor = 1.0e-6, 
        orient = False
    )

In [7]:
# df operations

df = (
    pd.read_csv("../../data/meteorite_landings.csv", quotechar='"', thousands=",")
    .dropna()
    .astype({"year": int})
)


df[["lon", "lat"]] = (
    df["GeoLocation"]
    .str.replace("(", "")
    .str.replace(")", "")
    .str.split(",", expand=True)
    .astype(float)
)

# convert coordinate to cartesian
points = to_cartesian(lons=df.lon, lats=df.lat)
mesh = points_to_mesh(points, df)

# plot the mesh
plotter = gv.GeoPlotter()
sargs = {"title": "Magnitude", "shadow": True}
kwargs = dict(
    scalar_bar_args=sargs,
    name  ="points",
    color ="red",
)

plotter.add_mesh(mesh, **kwargs)

def callback(value):
    mask = df["year"] == int(value)
    tmp = df[mask]
    if not tmp.empty:
        points = to_cartesian(tmp.lon, tmp.lat)
        mesh = points_to_mesh(points, tmp)
        plotter.add_mesh(mesh, **kwargs)
    else:
        plotter.remove_actor("points")

C:\Users\jivey\AppData\Local\Temp\ipykernel_21008\3452382124.py:31: UserWarning: Found no coordinate reference system (CRS) attached to mesh.
  plotter.add_mesh(mesh, **kwargs)


In [8]:
# plotter calls
min = int(df["year"].min())
max = int(df["year"].max())
plotter.add_slider_widget(callback, [min, max], interaction_event="always", fmt="%.0f")
plotter.add_base_layer(texture=gv.natural_earth_1(), zlevel=0)
plotter.add_graticule()
plotter.add_coastlines()
plotter.add_axes()
plotter.view_xz(negative=True)
plotter.camera.zoom(1.5)
plotter.show()

Decompressing 'C:\Users\jivey\AppData\Local\geovista\2023.08.0\mesh\lfric_c96.vtk.bz2' to 'C:\Users\jivey\AppData\Local\geovista\2023.08.0\mesh\lfric_c96.vtk' using method 'auto'.
Decompressing 'C:\Users\jivey\AppData\Local\geovista\2023.08.0\natural_earth\physical\ne_coastlines_10m.vtk.bz2' to 'C:\Users\jivey\AppData\Local\geovista\2023.08.0\natural_earth\physical\ne_coastlines_10m.vtk' using method 'auto'.


Widget(value="<iframe src='http://localhost:54299/index.html?ui=P_0x17261c28a30_0&reconnect=auto' style='width…